# How to kick off background runs

This guide covers how to kick off background runs for your agent.
This can be useful for long running jobs.

In [1]:
# Initialize the client
from langgraph_sdk import get_client

client = get_client()

In [2]:
# List available assistants
assistants = await client.assistants.search()
assistants[0]

{'assistant_id': 'e90fee30-be91-43aa-a33c-d54bd219072e',
 'graph_id': 'agent',
 'created_at': '2024-06-18T18:06:55.102231+00:00',
 'updated_at': '2024-06-18T18:06:55.102231+00:00',
 'config': {'configurable': {'model_name': 'anthropic'}},
 'metadata': {}}

In [3]:
# NOTE: we can use `assistant_id` UUID from the above response, or just pass graph ID instead when creating runs. we'll use graph ID here
assistant_id = "agent"

In [4]:
# Create a new thread
thread = await client.threads.create()
thread

{'thread_id': '5fc20631-47b7-48cd-8aa2-9f2eace9778d',
 'created_at': '2024-06-21T14:58:02.079462+00:00',
 'updated_at': '2024-06-21T14:58:02.079462+00:00',
 'metadata': {}}

In [5]:
# If we list runs on this thread, we can see it is empty
runs = await client.runs.list(thread['thread_id'])
runs

[]

In [6]:
# Let's kick off a run
input = {"messages": [{"role": "human", "content": "what's the weather in sf"}]}
run = await client.runs.create(thread['thread_id'], assistant_id, input=input)

In [7]:
# The first time we poll it, we can see `status=pending`
await client.runs.get(thread['thread_id'], run['run_id'])

{'run_id': '1ef2fdea-814c-6165-8b2a-a40e2a028198',
 'thread_id': '5fc20631-47b7-48cd-8aa2-9f2eace9778d',
 'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
 'created_at': '2024-06-21T14:58:02.095911+00:00',
 'updated_at': '2024-06-21T14:58:02.095911+00:00',
 'metadata': {},
 'status': 'pending',
 'kwargs': {'input': {'messages': [{'role': 'human',
     'content': 'what's the weather in sf'}]},
  'config': {'metadata': {'created_by': 'system'},
   'configurable': {'run_id': '1ef2fdea-814c-6165-8b2a-a40e2a028198',
    'user_id': '',
    'graph_id': 'agent',
    'thread_id': '5fc20631-47b7-48cd-8aa2-9f2eace9778d',
    'thread_ts': None,
    'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca'}},
  'webhook': None,
  'temporary': False,
  'stream_mode': ['events'],
  'feedback_keys': None,
  'interrupt_after': None,
  'interrupt_before': None},
 'multitask_strategy': 'reject'}

In [8]:
# Wait until the run finishes
await client.runs.join(thread['thread_id'], run['run_id'])

In [9]:
# Eventually, it should finish and we should see `status=success`
await client.runs.get(thread['thread_id'], run['run_id'])

{'run_id': '1ef2fdea-814c-6165-8b2a-a40e2a028198',
 'thread_id': '5fc20631-47b7-48cd-8aa2-9f2eace9778d',
 'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
 'created_at': '2024-06-21T14:58:02.095911+00:00',
 'updated_at': '2024-06-21T14:58:02.095911+00:00',
 'metadata': {},
 'status': 'success',
 'kwargs': {'input': {'messages': [{'role': 'human',
     'content': 'what's the weather in sf'}]},
  'config': {'metadata': {'created_by': 'system'},
   'configurable': {'run_id': '1ef2fdea-814c-6165-8b2a-a40e2a028198',
    'user_id': '',
    'graph_id': 'agent',
    'thread_id': '5fc20631-47b7-48cd-8aa2-9f2eace9778d',
    'thread_ts': None,
    'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca'}},
  'webhook': None,
  'temporary': False,
  'stream_mode': ['events'],
  'feedback_keys': None,
  'interrupt_after': None,
  'interrupt_before': None},
 'multitask_strategy': 'reject'}

In [10]:
# We can get the final results
final_result = await client.threads.get_state(thread['thread_id'])

In [11]:
final_result

{'values': {'messages': [{'content': 'what's the weather in sf',
    'additional_kwargs': {},
    'response_metadata': {},
    'type': 'human',
    'name': None,
    'id': 'bfe07fff-cb40-40be-84d5-a061d2c40006',
    'example': False},
   {'content': [{'id': 'toolu_01QUzhhfDQkpbPSediUrXvQb',
      'input': {'query': 'weather in san francisco'},
      'name': 'tavily_search_results_json',
      'type': 'tool_use'}],
    'additional_kwargs': {},
    'response_metadata': {},
    'type': 'ai',
    'name': None,
    'id': 'run-6d8665ca-a77d-4b44-9a7b-4e975b155fb1',
    'example': False,
    'tool_calls': [{'name': 'tavily_search_results_json',
      'args': {'query': 'weather in san francisco'},
      'id': 'toolu_01QUzhhfDQkpbPSediUrXvQb'}],
    'invalid_tool_calls': [],
    'usage_metadata': None},
   {'content': '[{"url": "https://www.timeanddate.com/weather/usa/san-francisco/historic", "content": "San Francisco Weather History for the Previous 24 Hours Show weather for: Previous 24 hours

In [12]:
# We can get the content of the final message
final_result['values']['messages'][-1]['content']

'The updated search provides the current weather forecast for San Francisco. According to the results, as of 7:49am on June 21, 2024 in San Francisco, the temperature is 55°F (12.8°C), it is overcast with 100% cloud cover, and there are light winds from the south-southwest around 7 mph (11 km/h). The forecast also shows low precipitation of 0.01 mm, high humidity of 86%, and visibility of 9 miles (16 km).\n\nIn summary, the current weather in San Francisco is cool, overcast, and breezy based on this weather forecast data. Let me know if you need any other details!'